In [31]:
#загрузим основные библиотеки
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
 
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import make_column_transformer
import joblib

In [32]:
# ЗАгрузим данные в датасет 
df = pd.read_csv('../data/Red.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8666 entries, 0 to 8665
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             8666 non-null   object 
 1   Country          8666 non-null   object 
 2   Region           8666 non-null   object 
 3   Winery           8666 non-null   object 
 4   Rating           8666 non-null   float64
 5   NumberOfRatings  8666 non-null   int64  
 6   Price            8666 non-null   float64
 7   Year             8666 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 541.8+ KB


In [33]:
# Создадим вектор с ответами и матрицу с признакми
X = df[['Region', 'Price', 'Country']]
y = df['Rating']

In [34]:
# Создадим перекодировку столбцов
ct = make_column_transformer(
    (OrdinalEncoder(), ['Region']),
    (StandardScaler(), ['Price']),
    (OneHotEncoder(), ['Country']))
print(ct)

ColumnTransformer(transformers=[('ordinalencoder', OrdinalEncoder(),
                                 ['Region']),
                                ('standardscaler', StandardScaler(), ['Price']),
                                ('onehotencoder', OneHotEncoder(),
                                 ['Country'])])


In [35]:
# Создадим пайплайн
pipeline = Pipeline([('ct', ct), ('rf', RandomForestRegressor(random_state=42))])


# Обучим модель по нашему пайплайну
pipeline.fit(X, y)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['Region']),
                                                 ('standardscaler',
                                                  StandardScaler(), ['Price']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Country'])])),
                ('rf', RandomForestRegressor(random_state=42))])

In [36]:
# Создадим функцию rmse
def rmse(y_hat, y):
    return mean_squared_error(y_hat, y, squared = False)

In [37]:
# создадим предсказания по тестовой выборке и вычислим rmse
data = pd.read_csv('../data/Red_test.csv')
X_test = data[['Country', 'Price', 'Region']]
y_test = data['Rating']
y_pred = pipeline.predict(X_test)
print(f'Качество по метрике R2: {round(r2_score(y_test, y_pred),4)}')
print(f'Качество по RSME: {round(rmse(y_test, y_pred),4)}')

joblib.dump(pipeline, 'practice_pipline.pkl')

Качество по метрике R2: 0.9394
Качество по RSME: 0.0764


['practice_pipline.pkl']

In [38]:
pipeline.set_params(rf__n_estimators=200)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['Region']),
                                                 ('standardscaler',
                                                  StandardScaler(), ['Price']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Country'])])),
                ('rf',
                 RandomForestRegressor(n_estimators=200, random_state=42))])

In [39]:
pipeline.fit(X,y)
y_pred = pipeline.predict(X_test)
print(f'Качество по RSME: {round(rmse(y_test, y_pred),4)}')

Качество по RSME: 0.076


In [40]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.tree import DecisionTreeRegressor

# Создаем список кортежей вида: (наименование модели, модель)
estimators = [
    ('lr', RidgeCV()),
    ('dt',  DecisionTreeRegressor(random_state=42))
]

# Создаем объект класса стекинг
reg = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor(n_estimators=10,
                                          random_state=42)
)
 
# Создадим пайплайн
pipeline_2 = Pipeline([('ct', ct), ('StackingRegressor', reg)])


# Обучим модель по нашему пайплайну
pipeline_2.fit(X, y)

y_pred = pipeline_2.predict(X_test)
print(f'Качество по RSME: {round(rmse(y_test, y_pred),2)}')

Качество по RSME: 0.18
